In [1]:
!pip install sentence_transformers 
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 5.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125926 sha256=3e162d62456c6a3a94a006c8c62386b1271653cb7581bce385e3938a14154073
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence_transformers
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 69.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.8/304.8 kB 29.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.7/311.7 kB 30.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 381.9/381.9 kB 36.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━

In [2]:
!pip install thefuzz

In [3]:
!pip install langdetect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 30.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993224 sha256=3c92b3a62640855af2d11bdf5b48c7ed3ac18506e1b7381437560a10d956f433
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect


In [5]:
from sentence_transformers import SentenceTransformer, util, InputExample, losses
from sentence_transformers.datasets import NoDuplicatesDataLoader
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu'
from tqdm.notebook import tqdm
import json
import os
import numpy as np
import random as rd

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# Load pre-trained model

model_name = 'imta-ai/multilingual-tech'
model = SentenceTransformer(model_name).to(device)

model.max_seq_length = 256

In [6]:
model_name = 'vietdata/sbert_qa'
device = 'cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu'
model = SentenceTransformer(model_name).to(device)

.gitattributes:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/2.96k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/22.0 [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

...

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/167 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.14k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/895k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

In [7]:
model.max_seq_length = 256

# Create Dataloader

In [8]:
from thefuzz import fuzz

In [9]:
with open("/kaggle/input/viblo-post/cleanBlogs.json",encoding="utf-8") as f:
    data = json.load(f)

In [10]:
print(data[0]["data"]["code"])

[{'head': 'Coffee', 'code': 'async_func = ->\n    d = Q.defer()\n    console.log("M: Should I?")\n    # Do something async\n    setTimeout((->\n        console.log("M: OK, give him")\n        d.resolve("M: Here\'re all mine")\n    ), 10000)\n    console.log("M: Ahhh, wait for my thinking. I promise!")\n    d.promise\n\nmain_func = ->\n    console.log("T: Free hand! Give me your money!")\n    console.log("T: You have 10 secs to think...")\n    async_func().then((async_promise)->\n        console.log(async_promise)\n    ).done((->\n        console.log("T: OK, you\'re dead! BAM")\n    ))\n    console.log("Hmmm?")\n    setTimeout((->\n        console.log("T: My gun is waiting")\n    ), 5000)\n\nmain_func()\n```'}, {'head': 'text', 'code': "> T: Free hand! Give me your money!\n> T: You have 10 secs to think...\n> M: Should I?\n> M: Ahhh, wait for my thinking. I promise!\n> Hmmm?\n> T: My gun is waiting\n> M: OK, give him\n> M: Here's all mine\n> T: OK, you're dead! BAM\n```"}]


In [11]:
data[5]["data"]["text"]

[{'content': '1 . Mức độ ưu tiên của id và class trong css . Một kiến thức khá đơn giản với những người làm fontend , nhưng không phải tất cả những dev bình thường đều biết . Đó là css dành sự ưu tiên nhiều hơn cho id . Khi một thẻ có cả id và class , thì những thuộc tính css viết cho id sẽ được ưu tiên . Ví dụ một thẻ p , có id là “ title ” , class “ text ” .',
  'length': 89},
 {'content': 'Có css như sau : Khi đó chữ trong thẻ p sẽ là chứ màu đen chứ không phải chứ màu đỏ . 2 .',
  'length': 25},
 {'content': 'Đặt tên có ý nghĩa cho ID và Class . Khi đặt một id hay class cho 1 thẻ html nào đó . Bạn nên đặt 1 cái tên có ý nghĩa , phù hợp nhất với tính năng . Như thế vừa giúp ta dễ hiểu , dễ quản lý , sau này cần sửa cũng dễ tìm mà còn tránh bị nhầm lẫn hay trùng lặp . Ví dụ trong 1 trang có nhiều button clear , một nút dùng để clear toàn bộ dữ liệu đã nhập , một nút lại chỉ clear dữ liệu ở một mục nhất định ( giả sử là giáo dục ) .',
  'length': 111},
 {'content': 'Nếu ta chỉ đặt cla

In [12]:
data[5]["data"]["title"]

'Một số chú ý dành cho dev backend khi viết css'

In [13]:
def create_pairs(title, contents, check=True):
    i = 0
    pairs = []
    while i < len(contents):
        increment = rd.choice([1, 2,3,4])
        content = "".join(contents[i:i+increment])
        i += increment
        if rd.random() < 0.5:
            title_ = title.lower()
        else:
            title_ = title
        
        if rd.random() < 0.5:
            content = content.lower()
        else:
            content = content
        if check and fuzz.token_sort_ratio(title_, content) > 80:
            pairs.append((title_, content))
        else:
            pairs.append((title_, content))
    return pairs

In [14]:
data[0]["data"]["title"]

'JavaScript Asynchronous: Promise by Q'

In [15]:
from langdetect import detect
data_ = []
for d in data:
    try:
        if detect(d["data"]["title"]) == "vi":
            data_.append(d)
    except:
        continue
len(data_)

25318

In [16]:
data = data_

In [17]:
pairs = []
for d in data:
    d = d["data"]
    title = d["title"]
    if "text" in d:
        pairs.extend(create_pairs(title, [j["content"] for j in d["text"]]))
#     if "code" in d:
#         pairs.extend(create_pairs(title, [j["code"] for j in d["code"]], check=False))

In [18]:
len(pairs)

190904

In [19]:
pairs[25]

('ứng dụng công nghệ phần mềm hướng tác tử trong thương mại điện tử (phần 2)',
 'trong phần 1 , tôi đã thực hiện tìm hiểu về công nghệ phần mềm hướng tác tử , các bạn có thể xem tại : nội dung tiếp theo của phần 2 sẽ là đàm phán trong thương mại điện tử và bước đầu xây dựng mô hình đàm phán trong thương mại điện tử . phần 2 : đàm phán và xây dựng mô hình đàm phán trong thương mại điện tử . đàm phán là một công việc quan trọng trong quá trình kinh doanh .hiện nay , các hệ thống tmđt thường không cung cấp cho người dùng khả năng đàm phán về giá cả và điều kiện mua bán . sau khi tìm kiếm thông tin về hàng hóa và lựa chọn nhà cung cấp nào đưa ra giá cả rẻ nhất , khách hàng sẽ tiến hành mua bán với giá cả và điều kiện cố định do nhà cung cấp đưa ra . một số thị trường điện tử có cung cấp cơ chế xác định giá cả thông qua các cuộc bán đấu giá trực tuyến . tuy nhiên hình thức này không thể thay thế cho mọi trường hợp đàm phán trong giao dịch thương mại .1 . một số định nghĩa . đàm phán : là mộ

In [20]:
def aug_length(texts):
    texts = [i["content"] for i in texts]
    i = 0
    ntexts = []
    while i < len(texts):
        increment = rd.choice([1, 2,3,4])
        content = "".join(texts[i:i+increment])
        i += increment
        ntexts.append(content)
    return ntexts

In [21]:
sentences = []
for d in data:
    if "text" in d["data"]:
        sentences.extend([j for j in aug_length(d["data"]["text"])])
#     if "code" in d["data"]:
#         sentences.extend([j["code"] for j in d["data"]["code"]])

In [22]:
sentences = [i for i in sentences if len(i) > 50]

In [23]:
sentences = list(set(sentences))
len(sentences)

187047

In [24]:
sentences[:5]

['1 . BDD là gì ? Phát triển hướng theo hành vi ( BDD ) là một hoạt động phát triển phần mềm làm việc trong một vòng lặp phản hồi ngắn , nơi chúng ta luôn áp dụng phát triển thử nghiệm cho mọi tính năng mới mà chúng ta đang khám phá và làm việc . Để có phần giới thiệu chi tiết về thực tiễn của BDD , bạn nên đọc bài viết trước của tôi về phát triển dựa vào hành vi ( Behavior-Driven Development ) .Bài viết này tập trung vào việc áp dụng các nguyên tắc này trong thực tiễn phát triển một ứng dụng web sử dụng Ruby on Rails . Như bài viết trước thì mình đã giới thiệu tổng quan cho các bạn thế nào là BDD , tác dụng của BDD còn nay thì sẽ tiếp tục nói về cách áp dụng BDD nhé ! Tool sử dụng đó là Cucumber . 2 . Công cụ viết BDD ( Cucumber ) cho ứng dụng Rails .',
 'Hỏi sâu vào định hướng . Sau câu số 2 nếu đã có định hướng cho bản thân ( giả sử bạn phỏng vấn cho vị trí Frontend Dev và định hướng Fullstack ) , thì anh phỏng vấn thường sẽ hỏi thêm là tại sao em lại chọn Frontend trước mà không ph

In [25]:
len(pairs)

190904

In [26]:
both = [(q, q) for q in sentences]+pairs
len(both)

377951

In [27]:
rd.shuffle(both)
both = both[:int(len(both)*0.5)]
len(both)

188975

In [28]:
import pandas as pd

df = pd.read_csv("/kaggle/input/vietnamese-quora-qa/quora_qa.csv")

In [29]:
both.extend(df.values.tolist())

In [30]:
len(both)

223985

In [31]:
train_data = [InputExample(texts=[q[0],q[1]]) for q in both]
train_dataloader = DataLoader(train_data, shuffle=True, batch_size=16,num_workers=os.cpu_count())
train_loss = losses.MultipleNegativesRankingLoss(model=model)

In [32]:
len(train_data)

223985

def sup_dataset(question):
    return [InputExample(texts=[q[0],q[1]], label=1.) for q in question if None not in q]

train_data_sup = sup_dataset(pairs)
sup_loader = DataLoader(train_data_sup, batch_size=8, shuffle=True,num_workers=os.cpu_count())

sup_loss = losses.CosineSimilarityLoss(model)

len(train_dataloader)

len(sup_loader)

for name, param in model.named_parameters():
    if "layer_norm" in name or "Norm" in name or "value" in name or "output.dense" in name or "intermediate.dense" in name:
        param.requires_grad = False

# Training

In [33]:
import traceback

try:
    model.fit(
        train_objectives=[(train_dataloader, train_loss)],#, (sup_loader, sup_loss)],
        epochs=3,
        warmup_steps=100,
        optimizer_params={'lr': 2e-5},
        output_path='./output/',
        show_progress_bar=True,
    )
except Exception as e:
    print(e)
    print(traceback.format_exc())
    pass

Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Iteration:   0%|          | 0/14000 [00:00<?, ?it/s]

Caught TypeError in DataLoader worker process 1.
Original Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/torch/utils/data/_utils/worker.py", line 308, in _worker_loop
    data = fetcher.fetch(index)
  File "/opt/conda/lib/python3.10/site-packages/torch/utils/data/_utils/fetch.py", line 54, in fetch
    return self.collate_fn(data)
  File "/opt/conda/lib/python3.10/site-packages/sentence_transformers/SentenceTransformer.py", line 551, in smart_batching_collate
    tokenized = self.tokenize(texts[idx])
  File "/opt/conda/lib/python3.10/site-packages/sentence_transformers/SentenceTransformer.py", line 319, in tokenize
    return self._first_module().tokenize(texts)
  File "/opt/conda/lib/python3.10/site-packages/sentence_transformers/models/Transformer.py", line 102, in tokenize
    batch1.append(text_tuple[0])
TypeError: 'float' object is not subscriptable

Traceback (most recent call last):
  File "/tmp/ipykernel_26/129743977.py", line 4, in <module>


In [34]:
model.save('./model/')

In [35]:
from huggingface_hub import HfApi
api = HfApi()

api.upload_folder(
    folder_path="/kaggle/working/model",
    repo_id="imta-ai/tech_sbert",
    repo_type="model",
)

pytorch_model.bin:   0%|          | 0.00/540M [00:00<?, ?B/s]

'https://huggingface.co/imta-ai/tech_sbert/tree/main/'

In [36]:
from huggingface_hub import snapshot_download
snapshot_download(repo_id="imta-ai/tech_sbert",cache_dir="/kaggle/working/", revision="d2aef7348b4b35672372c3be8b6b2a30a6c8286b")

Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

.gitattributes:   0%|          | 0.00/1.57k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/22.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/895k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/540M [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

'/kaggle/working/models--imta-ai--tech_sbert/snapshots/d2aef7348b4b35672372c3be8b6b2a30a6c8286b'

In [37]:
from transformers import AutoTokenizer
import torch
from sentence_transformers import SentenceTransformer, util, InputExample, losses
import numpy as np

In [38]:
model_name = "vietdata/sbert_qa"#'imta-ai/tech_sbert'
#model_name = "/kaggle/working/models--imta-ai--tech_sbert/snapshots/d2aef7348b4b35672372c3be8b6b2a30a6c8286b"
device = 'cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu'
model = SentenceTransformer(model_name).to(device)

In [39]:
import json

with open("/kaggle/input/kteam-course/course.json") as f:
    course = json.load(f)

In [40]:
course["data"][0]

"Khóa học lập trình C# căn bản , Bắt đầu học Khóa học lập trình C# căn bản //window.addEventListener('load', function () { // (adsbygoogle = window.adsbygoogle || []).push({}); //}); (adsbygoogle = window.adsbygoogle || []).push({}); Giới thiệu khóa học Bạn mới bắt đầu học lập trình? Bạn đang muốn học thêm ngôn ngữ lập trình mới? C# là lựa chọn hoàn hảo để đáp ứng các nhu cầu trên. Ngôn ngữ C# là một ngôn ngữ mới, cấu trúc rõ ràng, dễ hiểu và dễ học. C# thừa hưởng những ưu việt từ ngôn ngữ Java, C, C++ cũng như khắc phục được những hạn chế của các ngôn ngữ này. C# là ngôn ngữ lập trình hướng đối tượng được phát triển bởi Microsoft, được xây dựng dựa trên C++ và Java. Khoá học lần này sẽ mang đến toàn bộ những kiến thức cơ bản về C#. Chào mừng các bạn đã đến với khoá học LẬP TRÌNH C# CƠ BẢN của Kteam. Tham gia đóng góp khóa học cộng đồng Nếu bạn muốn gửi đến cộng đồng những khóa học"

In [41]:
sentences = [i for i in course["data"]]
question_embeddings = model.encode(sentences)

Batches:   0%|          | 0/6 [00:00<?, ?it/s]

In [42]:
sentences[0]

"Khóa học lập trình C# căn bản , Bắt đầu học Khóa học lập trình C# căn bản //window.addEventListener('load', function () { // (adsbygoogle = window.adsbygoogle || []).push({}); //}); (adsbygoogle = window.adsbygoogle || []).push({}); Giới thiệu khóa học Bạn mới bắt đầu học lập trình? Bạn đang muốn học thêm ngôn ngữ lập trình mới? C# là lựa chọn hoàn hảo để đáp ứng các nhu cầu trên. Ngôn ngữ C# là một ngôn ngữ mới, cấu trúc rõ ràng, dễ hiểu và dễ học. C# thừa hưởng những ưu việt từ ngôn ngữ Java, C, C++ cũng như khắc phục được những hạn chế của các ngôn ngữ này. C# là ngôn ngữ lập trình hướng đối tượng được phát triển bởi Microsoft, được xây dựng dựa trên C++ và Java. Khoá học lần này sẽ mang đến toàn bộ những kiến thức cơ bản về C#. Chào mừng các bạn đã đến với khoá học LẬP TRÌNH C# CƠ BẢN của Kteam. Tham gia đóng góp khóa học cộng đồng Nếu bạn muốn gửi đến cộng đồng những khóa học"

In [43]:
def get_answer(question):
    user_question_embedding = model.encode(question)
    scores = util.semantic_search(user_question_embedding, question_embeddings)[0]
    scores = np.array(scores).flatten()
    #scores = np.concatenate([scores1, scores2])
    scores = sorted(scores, reverse=True, key=lambda x: x['score'])
    idx = scores[0]['corpus_id']
    return [(i["score"], sentences[:10000][i["corpus_id"]]) for i in scores[:10]]

In [44]:
get_answer("học react làm app ")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

[(0.5709726214408875,
  'Kỹ thuật Cracking , file crack app Buổi 2: Tạo script crack hàng loạt file easy Buổi 3: Hiển thị MessageBox lên Calculator Đôi nét về host | | | | --- | --- | | | Trực tiếp tham gia giao lưu và hướng dẫn các bạn trong chuỗi livestream kỹ thuật này chính là anh Kim Long * CEO Kteam Co * Founder Howkteam.vn, Kquiz.vn * Đồng thời là tác giả quen thuộc của rất nhiều khóa học về C#, python, Java, Auto,... tại Howkteam | Mong rằng sẽ giúp các bạn có được những bước đầu tiên, thật cơ bản về mảng kỹ thuật máy tính. Và phần nào định hướng cho các bạn newbie biết về một khía cạnh phát triển khác của ngành CNTT; ngoài các ứng dụng quen thuộc như web, app, game,...'),
 (0.5695300102233887,
  "Khóa học JavaScript cơ bản , Bắt đầu học Khóa học JavaScript cơ bản //window.addEventListener('load', function () { // (adsbygoogle = window.adsbygoogle || []).push({}); //}); (adsbygoogle = window.adsbygoogle || []).push({}); Giới thiệu khóa học JavaScript là một trong những ngôn ngữ